In [85]:
library(tidyverse)

# Custom package
library(rutils)

In [86]:
dirs <- rutils::get_dev_directories(dev_paths_file = "../dev_paths.txt")
projects <- c("TCGA-CESC", "TCGA-UCS", "TCGA-UCEC", "TCGA-OV")
unified_dsets <- c("unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data")
matrisome_list <- paste0(dirs$data_dir, "/matrisome/matrisome_hs_masterlist.tsv")

In [87]:
dset_idx <- 1

In [88]:
p_thresh <- 0.05
lfc_thresh <- log2(2)
coxph_coeff_thresh <- 0.0
mi_thresh <- 0.0
consensus_thresh <- 0.0
consensus_n <- 5
hub_con_thresh <- 0

# Helpers

In [89]:
matrisome_df <- rutils::load_matrisome_df(matrisome_list) %>%
    dplyr::select(gene_symbol, division, category)


get_consensus_col <- function(df, n, thresh = 0) {
    consensus_df <- df %>%
        dplyr::select(geneID, contains("mean")) %>%
        dplyr::mutate(consensus = rowSums(.[-1] > thresh) >= n)
    return(df %>% dplyr::mutate(consensus = consensus_df$consensus))
}


-- Column specification ------------------------------------------------------------------------------------------------
cols(
  Division = col_character(),
  Category = col_character(),
  `Gene Symbol` = col_character(),
  `Gene Name` = col_character(),
  Synonyms = col_character(),
  HGNC_IDs = col_double(),
  `HGNC_IDs Links` = col_double(),
  UniProt_IDs = col_character(),
  Refseq_IDs = col_character(),
  Orthology = col_character(),
  Notes = col_character()
)




In [90]:
norm_counts_df <- read_tsv(paste0(dirs$data_dir, "/", unified_dsets[dset_idx], "/", "norm_counts.tsv"))


-- Column specification ------------------------------------------------------------------------------------------------
cols(
  .default = col_double(),
  geneID = col_character()
)
i Use `spec()` for the full column specifications.




# DGE analysis

In [91]:
DESeq_results_df <- read_tsv(paste0(dirs$analysis_dir, "/deg/", unified_dsets[dset_idx], "_DESeq_results.tsv"))
# filtered_DESeq_results_df <- DESeq_results_df %>%
#     dplyr::filter(abs(log2FoldChange) > lfc_thresh, padj < p_thresh)
filtered_DESeq_results_df <- DESeq_results_df %>%
    dplyr::filter(abs(log2FoldChange) > lfc_thresh, qval < p_thresh)

filtered_matrisome_DESeq_results_df <- filtered_DESeq_results_df %>%
    dplyr::filter(geneID %in% matrisome_df$gene_symbol)

n_degs <- nrow(filtered_DESeq_results_df)
deg_prop <- nrow(filtered_DESeq_results_df) / nrow(norm_counts_df)
n_degs_up <- nrow(filtered_DESeq_results_df %>% dplyr::filter(log2FoldChange > 0))
n_degs_down <- nrow(filtered_DESeq_results_df %>% dplyr::filter(log2FoldChange < 0))

n_matrisome_degs <- nrow(filtered_matrisome_DESeq_results_df)
matrisome_deg_prop <- nrow(filtered_matrisome_DESeq_results_df) / nrow(matrisome_df)
n_matrisome_degs_up <- nrow(filtered_matrisome_DESeq_results_df %>% dplyr::filter(log2FoldChange > 0))
n_matrisome_degs_down <- nrow(filtered_matrisome_DESeq_results_df %>% dplyr::filter(log2FoldChange < 0))


paste0("DEGs: ", n_degs)
paste0("DEG prop: ", deg_prop)
paste0("Up: ", n_degs_up)
paste0("Down: ", n_degs_down)
paste0("Matrisome DEGs: ", n_matrisome_degs)
paste0("Matrisome DEG prop: ", matrisome_deg_prop)
paste0("Matrisome up: ", n_matrisome_degs_up)
paste0("Matrisome down: ", n_matrisome_degs_down)


-- Column specification ------------------------------------------------------------------------------------------------
cols(
  geneID = col_character(),
  baseMean = col_double(),
  log2FoldChange = col_double(),
  lfcSE = col_double(),
  stat = col_double(),
  pvalue = col_double(),
  padj = col_double(),
  qval = col_double()
)




[1] "DEGs: 7652"

[1] "DEG prop: 0.378025886770082"

[1] "Up: 3966"

[1] "Down: 3686"

[1] "Matrisome DEGs: 593"

[1] "Matrisome DEG prop: 0.577409931840312"

[1] "Matrisome up: 262"

[1] "Matrisome down: 331"

# Survival analysis

In [92]:
coxph_null_scores_df <- read_tsv(paste0(dirs$analysis_dir, "/meta/", "coxph_null_scores.tsv"))
coxph_results_df <- read_tsv(paste0(dirs$analysis_dir, "/feature_selection/", unified_dsets[dset_idx], "_coxph_results.tsv"))
# filtered_coxph_results_df <- coxph_results_df %>%
#     dplyr::filter(gene_pval < p_thresh)
filtered_coxph_results_df <- coxph_results_df %>%
    dplyr::filter(gene_qval < p_thresh)


-- Column specification ------------------------------------------------------------------------------------------------
cols(
  dataset = col_character(),
  lr_test_pval = col_double(),
  wald_test_pval = col_double(),
  score_test_pval = col_double()
)



-- Column specification ------------------------------------------------------------------------------------------------
cols(
  geneID = col_character(),
  gene_pval = col_double(),
  gene_coeff = col_double(),
  gene_qval = col_double(),
  gene_padj = col_double()
)




In [93]:
coxph_null_scores_df

dataset,lr_test_pval,wald_test_pval,score_test_pval
<chr>,<dbl>,<dbl>,<dbl>
unified_cervical_data,0.0004814013,0.000049053,1.224173e-06
unified_uterine_data,0.3392807906,0.989893132,3.053979e-01
unified_uterine_endometrial_data,0.0007739010,0.000000000,1.040876e-06


In [94]:
coxph_null_sig <- (coxph_null_scores_df %>% dplyr::filter(dataset == unified_dsets[dset_idx]))$lr_test_pval < p_thresh
n_coxph_sig <- nrow(filtered_coxph_results_df)
prop_coxph_sig <- n_coxph_sig / nrow(matrisome_df)
n_coxph_sig_protective <- nrow(filtered_coxph_results_df %>% dplyr::filter(gene_coeff < 0))
n_coxph_sig_harmful <- nrow(filtered_coxph_results_df %>% dplyr::filter(gene_coeff > 0))


paste0("Sig. null Cox PH: ", coxph_null_sig)
paste0("Sig. Cox PH genes: ", n_coxph_sig)
paste0("Prop. sig. Cox PH genes: ", prop_coxph_sig)
paste0("Sig. Cox PH protective: ", n_coxph_sig_protective)
paste0("Sig. Cox PH harmful: ", n_coxph_sig_harmful)

[1] "Sig. null Cox PH: TRUE"

[1] "Sig. Cox PH genes: 55"

[1] "Prop. sig. Cox PH genes: 0.053554040895813"

[1] "Sig. Cox PH protective: 15"

[1] "Sig. Cox PH harmful: 40"

# Regression

## Correlation analysis

In [95]:
cor_results_df <- read_tsv(paste0(dirs$analysis_dir, "/feature_selection/", unified_dsets[dset_idx], "_cor_results.tsv"))

# filtered_cor_results_df <- cor_results_df %>% dplyr::filter(pval < p_thresh)
# filtered_cor_results_df <- cor_results_df %>% dplyr::filter(padj < p_thresh)
filtered_cor_results_df <- cor_results_df %>% dplyr::filter(qval < p_thresh)

n_cor <- nrow(filtered_cor_results_df)
n_cor_down <- nrow(filtered_cor_results_df %>% dplyr::filter(cor < 0))
n_cor_up <- nrow(filtered_cor_results_df %>% dplyr::filter(cor > 0))

paste0("Cor. genes: ", n_cor)
paste0("Neg. cor. genes: ", n_cor_down)
paste0("Pos. cor. genes: ", n_cor_up)


-- Column specification ------------------------------------------------------------------------------------------------
cols(
  geneID = col_character(),
  cor = col_double(),
  pval = col_double(),
  padj = col_double(),
  qval = col_double(),
  n = col_double()
)




[1] "Cor. genes: 1"

[1] "Neg. cor. genes: 0"

[1] "Pos. cor. genes: 1"

## MI analysis

In [96]:
mi_survival_results_df <- read_tsv(paste0(dirs$analysis_dir, "/feature_selection/", unified_dsets[dset_idx], "_MI_survival_results.tsv"))
filtered_mi_survival_results_df <- mi_survival_results_df %>% dplyr::filter(MI_est_median > mi_thresh)

n_mi <- nrow(filtered_mi_survival_results_df)

paste0("MI: ", n_mi)


-- Column specification ------------------------------------------------------------------------------------------------
cols(
  geneID = col_character(),
  MI_est_median = col_double()
)




[1] "MI: 550"

In [97]:
mi_survival_ordered_df <- mi_survival_results_df %>%
    dplyr::arrange(desc(MI_est_median)) %>%
    dplyr::mutate(geneID_f = factor(geneID, levels = geneID)) %>%
    dplyr::filter(MI_est_median > 0.0)
nrow(mi_survival_ordered_df %>%
    dplyr::mutate(pct_delta_max = (MI_est_median - first(MI_est_median)) / first(MI_est_median) * 100) %>%
    dplyr::filter(pct_delta_max > -50))

# ggplot(data = mi_survival_ordered_df) +
#     geom_bar(aes(x = geneID_f, y = MI_est_median), stat = "identity") +
#     theme_classic() +
#     theme(
#         axis.text.x = element_blank(),
#         axis.ticks.x = element_blank()
#     ) +
#     labs(x = "Matrisome gene", y = "Mutual information score")


[1] 45

## Baselines

In [98]:
reg_baselines_df <- read_tsv(paste0(dirs$analysis_dir, "/meta/", "reg_baselines.tsv"))
mse_baseline = (reg_baselines_df %>% filter(dataset == unified_dsets[dset_idx]))$L2
mae_baseline = (reg_baselines_df %>% filter(dataset == unified_dsets[dset_idx]))$L1
ev_baseline = (reg_baselines_df %>% filter(dataset == unified_dsets[dset_idx]))$explained_variance
paste0("MSE baseline: ", mse_baseline)
paste0("MAE baseline: ", mae_baseline)
paste0("EV baseline: ", ev_baseline)


-- Column specification ------------------------------------------------------------------------------------------------
cols(
  dataset = col_character(),
  L2 = col_double(),
  L1 = col_double(),
  R2 = col_double(),
  explained_variance = col_double(),
  n = col_double()
)




[1] "MSE baseline: 641687.698806244"

[1] "MAE baseline: 518.333333333333"

[1] "EV baseline: 0"

## GBR (MSE)

In [99]:
mse_gbr_scores_df <- read_tsv(paste0(dirs$analysis_dir, "/feature_selection/", unified_dsets[dset_idx], "_mse_gbr_ref_scores.tsv"))
mse_gbr_results_df <- read_tsv(paste0(dirs$analysis_dir, "/feature_selection/", unified_dsets[dset_idx], "_mse_gbr_results.tsv"))
mse_gbr_results_df <- get_consensus_col(mse_gbr_results_df, n = consensus_n, thresh = consensus_thresh)
mse_gbr_scores_df

# mult. by -1 to convert neg MSE -> MSE
mse_gbr_avg <- mean(-mse_gbr_scores_df$ref_score)
mse_gbr_imp <- mse_gbr_avg < mse_baseline
# mult. by -1 since pct. "improvement" is pct. reduction in this case
mse_gbr_baseline_pct_imp <- - (mse_gbr_avg - mse_baseline) / mse_baseline * 100
n_mse_gbr_consensus_genes <- nrow(mse_gbr_results_df %>% dplyr::filter(consensus == TRUE))

paste0("GBR MSE avg.: ", mse_gbr_avg)
paste0("GBR MSE impr. over baseline: ", mse_gbr_imp)
paste0("GBR MSE pct. impr. over baseline: ", mse_gbr_baseline_pct_imp)
paste0("GBR MSE consensus genes: ", n_mse_gbr_consensus_genes)


-- Column specification ------------------------------------------------------------------------------------------------
cols(
  model = col_double(),
  ref_score = col_double()
)



-- Column specification ------------------------------------------------------------------------------------------------
cols(
  geneID = col_character(),
  mean_imp_0 = col_double(),
  score_pct_improvement_0 = col_double(),
  mean_imp_1 = col_double(),
  score_pct_improvement_1 = col_double(),
  mean_imp_2 = col_double(),
  score_pct_improvement_2 = col_double(),
  mean_imp_3 = col_double(),
  score_pct_improvement_3 = col_double(),
  mean_imp_4 = col_double(),
  score_pct_improvement_4 = col_double()
)




model,ref_score
<dbl>,<dbl>
0,-636158.7
1,-634333.3
2,-634684.4
3,-698686.7
4,-634982.6


[1] "GBR MSE avg.: 647769.127646991"

[1] "GBR MSE impr. over baseline: FALSE"

[1] "GBR MSE pct. impr. over baseline: -0.947724080118784"

[1] "GBR MSE consensus genes: 78"

## RFR (MSE)

In [100]:
mse_rfr_scores_df <- read_tsv(paste0(dirs$analysis_dir, "/feature_selection/", unified_dsets[dset_idx], "_mse_rfr_ref_scores.tsv"))
mse_rfr_results_df <- read_tsv(paste0(dirs$analysis_dir, "/feature_selection/", unified_dsets[dset_idx], "_mse_rfr_results.tsv"))
mse_rfr_results_df <- get_consensus_col(mse_rfr_results_df, n = consensus_n, thresh = consensus_thresh)
mse_rfr_scores_df

# mult. by -1 to convert neg MSE -> MSE
mse_rfr_avg <- mean(-mse_rfr_scores_df$ref_score)
mse_rfr_imp <- mse_rfr_avg < mse_baseline
# mult. by -1 since pct. "improvement" is pct. reduction in this case
mse_rfr_baseline_pct_imp <- - (mse_rfr_avg - mse_baseline) / mse_baseline * 100
n_mse_rfr_consensus_genes <- nrow(mse_rfr_results_df %>% dplyr::filter(consensus == TRUE))

paste0("RFR MSE avg.: ", mse_rfr_avg)
paste0("RFR MSE impr. over baseline: ", mse_rfr_imp)
paste0("RFR MSE pct. impr. over baseline: ", mse_rfr_baseline_pct_imp)
paste0("RFR MSE consensus genes: ", n_mse_rfr_consensus_genes)


-- Column specification ------------------------------------------------------------------------------------------------
cols(
  model = col_double(),
  ref_score = col_double()
)



-- Column specification ------------------------------------------------------------------------------------------------
cols(
  geneID = col_character(),
  mean_imp_0 = col_double(),
  score_pct_improvement_0 = col_double(),
  mean_imp_1 = col_double(),
  score_pct_improvement_1 = col_double(),
  mean_imp_2 = col_double(),
  score_pct_improvement_2 = col_double(),
  mean_imp_3 = col_double(),
  score_pct_improvement_3 = col_double(),
  mean_imp_4 = col_double(),
  score_pct_improvement_4 = col_double()
)




model,ref_score
<dbl>,<dbl>
0,-655024.1
1,-646579.8
2,-634329.3
3,-640585.0
4,-645026.2


[1] "RFR MSE avg.: 644308.877471592"

[1] "RFR MSE impr. over baseline: FALSE"

[1] "RFR MSE pct. impr. over baseline: -0.408481987456537"

[1] "RFR MSE consensus genes: 53"

# Classification

## Anova analysis

In [101]:
welch_anova_results_df <- read_tsv(paste0(dirs$analysis_dir, "/feature_selection/", unified_dsets[dset_idx], "_welch_anova_results.tsv"))
# filtered_welch_anova_results_df <- welch_anova_results_df %>% dplyr::filter(padj < p_thresh)
filtered_welch_anova_results_df <- welch_anova_results_df %>% dplyr::filter(qval < p_thresh)

n_sig <- nrow(filtered_welch_anova_results_df)

paste0("Sig. ANOVA: ", n_sig)


-- Column specification ------------------------------------------------------------------------------------------------
cols(
  geneID = col_character(),
  pval = col_double(),
  padj = col_double(),
  qval = col_double()
)




[1] "Sig. ANOVA: 3"

## MI analysis

In [102]:
mi_figo_results_df <- read_tsv(paste0(dirs$analysis_dir, "/feature_selection/", unified_dsets[dset_idx], "_MI_figo_results.tsv"))
filtered_mi_figo_results_df <- mi_figo_results_df %>% dplyr::filter(MI_est_median > mi_thresh)

n_mi <- nrow(filtered_mi_figo_results_df)

paste0("MI: ", n_mi)


-- Column specification ------------------------------------------------------------------------------------------------
cols(
  geneID = col_character(),
  MI_est_median = col_double()
)




[1] "MI: 506"

In [103]:
mi_figo_ordered_df <- mi_figo_results_df %>%
    dplyr::arrange(desc(MI_est_median)) %>%
    dplyr::mutate(geneID_f = factor(geneID, levels = geneID)) %>%
    dplyr::filter(MI_est_median > 0.0)
nrow(mi_figo_ordered_df %>%
    dplyr::mutate(pct_delta_max = (MI_est_median - first(MI_est_median)) / first(MI_est_median) * 100) %>%
    dplyr::filter(pct_delta_max > -50))

# ggplot(data = mi_figo_ordered_df) +
#     geom_bar(aes(x = geneID_f, y = MI_est_median), stat = "identity") +
#     theme_classic() +
#     theme(
#         axis.text.x = element_blank(),
#         axis.ticks.x = element_blank()
#     ) +
#     labs(x = "Matrisome gene", y = "Mutual information score")

[1] 56

## Baselines

In [104]:
cls_baselines_df <- read_tsv(paste0(dirs$analysis_dir, "/meta/", "cls_baselines.tsv"))
cls_baselines_df
f1_macro_majority_baseline <- (cls_baselines_df %>% filter(dataset == unified_dsets[dset_idx]))$f1_macro_majority
f1_macro_MC_baseline <- (cls_baselines_df %>% filter(dataset == unified_dsets[dset_idx]))$f1_macro_MC


paste0("F1 macro (majority guess) baseline: ", f1_macro_majority_baseline)
paste0("F1 macro (Monte Carlo) baseline: ", f1_macro_MC_baseline)


-- Column specification ------------------------------------------------------------------------------------------------
cols(
  dataset = col_character(),
  f1_macro_majority = col_double(),
  f1_macro_MC = col_double(),
  n = col_double()
)




dataset,f1_macro_majority,f1_macro_MC,n
<chr>,<dbl>,<dbl>,<dbl>
unified_cervical_data,0.1730769,0.2487013,255
unified_uterine_data,0.1406250,0.2443586,46
unified_uterine_endometrial_data,0.1738095,0.2480810,137


[1] "F1 macro (majority guess) baseline: 0.173076923076923"

[1] "F1 macro (Monte Carlo) baseline: 0.248701296417504"

## GBC

In [105]:
f1_gbc_scores_df <- read_tsv(paste0(dirs$analysis_dir, "/feature_selection/", unified_dsets[dset_idx], "_gbc_ref_scores.tsv"))
f1_gbc_results_df <- read_tsv(paste0(dirs$analysis_dir, "/feature_selection/", unified_dsets[dset_idx], "_gbc_results.tsv"))
f1_gbc_results_df <- get_consensus_col(f1_gbc_results_df, n = consensus_n, thresh = consensus_thresh)
f1_gbc_scores_df

f1_gbc_avg <- mean(f1_gbc_scores_df$ref_score)
f1_gbc_imp <- f1_gbc_avg > f1_macro_MC_baseline
f1_gbc_baseline_pct_imp <- (f1_gbc_avg - f1_macro_MC_baseline) / f1_macro_MC_baseline * 100
n_f1_gbc_consensus_genes <- nrow(f1_gbc_results_df %>% dplyr::filter(consensus == TRUE))

paste0("GBC F1 avg.: ", f1_gbc_avg)
paste0("GBC F1 impr. over baseline: ", f1_gbc_imp)
paste0("GBC F1 pct. impr. over baseline: ", f1_gbc_baseline_pct_imp)
paste0("GBC F1 consensus genes: ", n_f1_gbc_consensus_genes)


-- Column specification ------------------------------------------------------------------------------------------------
cols(
  model = col_double(),
  ref_score = col_double()
)



-- Column specification ------------------------------------------------------------------------------------------------
cols(
  geneID = col_character(),
  mean_imp_0 = col_double(),
  score_pct_improvement_0 = col_double(),
  mean_imp_1 = col_double(),
  score_pct_improvement_1 = col_double(),
  mean_imp_2 = col_double(),
  score_pct_improvement_2 = col_double(),
  mean_imp_3 = col_double(),
  score_pct_improvement_3 = col_double(),
  mean_imp_4 = col_double(),
  score_pct_improvement_4 = col_double()
)




model,ref_score
<dbl>,<dbl>
0,0.3184352
1,0.3259052
2,0.3103286
3,0.2812384
4,0.3222672


[1] "GBC F1 avg.: 0.31163490763661"

[1] "GBC F1 impr. over baseline: TRUE"

[1] "GBC F1 pct. impr. over baseline: 25.3048987382267"

[1] "GBC F1 consensus genes: 43"

## RFC

In [106]:
f1_rfc_scores_df <- read_tsv(paste0(dirs$analysis_dir, "/feature_selection/", unified_dsets[dset_idx], "_rfc_ref_scores.tsv"))
f1_rfc_results_df <- read_tsv(paste0(dirs$analysis_dir, "/feature_selection/", unified_dsets[dset_idx], "_rfc_results.tsv"))
f1_rfc_results_df <- get_consensus_col(f1_rfc_results_df, n = consensus_n, thresh = consensus_thresh)
f1_rfc_scores_df

f1_rfc_avg <- mean(f1_rfc_scores_df$ref_score)
f1_rfc_imp <- f1_rfc_avg > f1_macro_MC_baseline
f1_rfc_baseline_pct_imp <- (f1_rfc_avg - f1_macro_MC_baseline) / f1_macro_MC_baseline * 100
n_f1_rfc_consensus_genes <- nrow(f1_rfc_results_df %>% dplyr::filter(consensus == TRUE))

paste0("rfc F1 avg.: ", f1_rfc_avg)
paste0("rfc F1 impr. over baseline: ", f1_rfc_imp)
paste0("rfc F1 pct. impr. over baseline: ", f1_rfc_baseline_pct_imp)
paste0("rfc F1 consensus genes: ", n_f1_rfc_consensus_genes)


-- Column specification ------------------------------------------------------------------------------------------------
cols(
  model = col_double(),
  ref_score = col_double()
)



-- Column specification ------------------------------------------------------------------------------------------------
cols(
  geneID = col_character(),
  mean_imp_0 = col_double(),
  score_pct_improvement_0 = col_double(),
  mean_imp_1 = col_double(),
  score_pct_improvement_1 = col_double(),
  mean_imp_2 = col_double(),
  score_pct_improvement_2 = col_double(),
  mean_imp_3 = col_double(),
  score_pct_improvement_3 = col_double(),
  mean_imp_4 = col_double(),
  score_pct_improvement_4 = col_double()
)




model,ref_score
<dbl>,<dbl>
0,0.2186327
1,0.2041763
2,0.2272414
3,0.1984757
4,0.2281829


[1] "rfc F1 avg.: 0.215341793755938"

[1] "rfc F1 impr. over baseline: FALSE"

[1] "rfc F1 pct. impr. over baseline: -13.4134816111149"

[1] "rfc F1 consensus genes: 8"

## LR (L1)

In [107]:
f1_l1_lr_scores_df <- read_tsv(paste0(dirs$analysis_dir, "/feature_selection/", unified_dsets[dset_idx], "_l1_lr_ref_scores.tsv"))
f1_l1_lr_results_df <- read_tsv(paste0(dirs$analysis_dir, "/feature_selection/", unified_dsets[dset_idx], "_l1_lr_results.tsv"))
f1_l1_lr_results_df <- get_consensus_col(f1_l1_lr_results_df, n = consensus_n, thresh = consensus_thresh)
f1_l1_lr_scores_df

f1_l1_lr_avg <- mean(f1_l1_lr_scores_df$ref_score)
f1_l1_lr_imp <- f1_l1_lr_avg > f1_macro_MC_baseline
f1_l1_lr_pct_imp <- (f1_l1_lr_avg - f1_macro_MC_baseline) / f1_macro_MC_baseline * 100
n_f1_l1_lr_consensus_genes <- nrow(f1_l1_lr_results_df %>% dplyr::filter(consensus == TRUE))

paste0("LR L1 F1 avg.: ", f1_l1_lr_avg)
paste0("LR L1 F1 impr. over baseline: ", f1_l1_lr_imp)
paste0("LR L1 F1 pct. impr. over baseline: ", f1_l1_lr_pct_imp)
paste0("LR L1 F1 consensus genes: ", n_f1_l1_lr_consensus_genes)


-- Column specification ------------------------------------------------------------------------------------------------
cols(
  model = col_double(),
  ref_score = col_double()
)



-- Column specification ------------------------------------------------------------------------------------------------
cols(
  geneID = col_character(),
  mean_imp_0 = col_double(),
  score_pct_improvement_0 = col_double(),
  mean_imp_1 = col_double(),
  score_pct_improvement_1 = col_double(),
  mean_imp_2 = col_double(),
  score_pct_improvement_2 = col_double(),
  mean_imp_3 = col_double(),
  score_pct_improvement_3 = col_double(),
  mean_imp_4 = col_double(),
  score_pct_improvement_4 = col_double()
)




model,ref_score
<dbl>,<dbl>
0,0.3348598
1,0.3347403
2,0.3371665
3,0.3365318
4,0.3365318


[1] "LR L1 F1 avg.: 0.335966034453971"

[1] "LR L1 F1 impr. over baseline: TRUE"

[1] "LR L1 F1 pct. impr. over baseline: 35.0881717520173"

[1] "LR L1 F1 consensus genes: 192"

# Network Analysis

In [108]:
network_mm_gs_df <- read_tsv(paste0(dirs$analysis_dir, "/network/", unified_dsets[dset_idx], "_gene_mm_gs.tsv"))
network_me_sig_df <- read_tsv(paste0(dirs$analysis_dir, "/network/", unified_dsets[dset_idx], "_eigengene_traits.tsv"))
lnames <- load(file = paste0(dirs$data_dir, "/saved_network_objects/", unified_dsets[dset_idx], "_tumor_data.RData"))
lnames <- load(file = paste0(dirs$data_dir, "/saved_network_objects/", unified_dsets[dset_idx], "_tumor_network.RData"))
pval_thresh <- 0.05

hub_df <- get_most_conn_genes(data_expr, module_colors, soft_power, conn_vs_hub_thresh = hub_con_thresh) %>%
    bind_rows(.id = "module") %>%
    dplyr::select(geneID, everything())



-- Column specification ------------------------------------------------------------------------------------------------
cols(
  .default = col_double(),
  geneID = col_character(),
  module = col_character()
)
i Use `spec()` for the full column specifications.



-- Column specification ------------------------------------------------------------------------------------------------
cols(
  module = col_character(),
  figo_stage_1_cor = col_double(),
  figo_stage_2_cor = col_double(),
  figo_stage_3_cor = col_double(),
  figo_stage_4_cor = col_double(),
  figo_stage_1_pval = col_double(),
  figo_stage_2_pval = col_double(),
  figo_stage_3_pval = col_double(),
  figo_stage_4_pval = col_double(),
  figo_stage_1_qval = col_double(),
  figo_stage_2_qval = col_double(),
  figo_stage_3_qval = col_double(),
  figo_stage_4_qval = col_double(),
  vital_hr = col_double(),
  vital_dev_cor = col_double(),
  vital_pval = col_double(),
  vital_qval = col_double()
)




In [109]:
condensed_me_df <- network_me_sig_df %>%
    condense_figo(include_pvals = TRUE) %>%
    dplyr::rename_if(!startsWith(colnames(.), "module"), ~ gsub("^", "me_", .))

filtered_figo_network_df <- network_mm_gs_df %>%
    dplyr::select(geneID, module, mm_pval, mm_cor) %>%
    inner_join(condensed_me_df, by = "module") %>%
    dplyr::filter(me_figo_min_qval < pval_thresh) %>%
    # Make sure genes are significant members of the module
    dplyr::filter(mm_pval < pval_thresh) %>%
    # Make sure genes are highly connected within the module
    dplyr::filter(geneID %in% hub_df$geneID)

filtered_coxph_network_df <- network_mm_gs_df %>%
    dplyr::select(geneID, module, mm_pval, mm_cor) %>%
    inner_join(condensed_me_df, by = "module") %>%
    dplyr::filter(me_vital_qval < pval_thresh) %>%
    # Make sure genes are significant members of the module
    dplyr::filter(mm_pval < pval_thresh) %>%
    # Make sure genes are highly connected within the module
    dplyr::filter(geneID %in% hub_df$geneID)

In [110]:
nrow(filtered_figo_network_df)
nrow(filtered_coxph_network_df)
length(unique(filtered_figo_network_df$module))
length(unique(filtered_coxph_network_df$module))

[1] 232

[1] 0

[1] 3

[1] 0

In [111]:
# Subtract 1 from diagonal of TOM (so we don't count self-TOM)
tom_summary_df <- tibble(geneID = colnames(data_expr), tom_conn = rowSums(tom - diag(1, nrow(tom), ncol(tom)))) %>%
    arrange(desc(tom_conn))
round(mean(tom_summary_df$tom_conn), 3)
round(quantile(tom_summary_df$tom_conn), 3)

[1] 16.387

0%    25%    50%    75%   100% 
 1.569  8.313 12.799 20.526 72.430